最終課題

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re
import random
from collections import deque

# --- 設定 ---
# 武蔵野大学のトップページURL 
START_URL = "https://www.musashino-u.ac.jp"
# 同一ドメインのチェック用
DOMAIN = urlparse(START_URL).netloc
# 最大ページ数制限 (Noneで無制限)
MAX_PAGES_LIMIT = None
# クロール負荷軽減のための遅延設定
# time.sleepの最小・最大遅延時間（秒）
MIN_DELAY = 3.0
MAX_DELAY = 5.0
# --- 

# サイトマップ格納用の辞書
sitemap = {}
# クロール対象のURLキュー (FIFO)
to_crawl = deque([START_URL])
# クロール済みのURL集合（to_crawlに追加済みも含めて重複チェックに使用）
processed_urls = {START_URL}

print(f"クロールを開始します: {START_URL}")
print(f"対象ドメイン: {DOMAIN}")
print(f"最大 {MAX_PAGES_LIMIT} ページまでクロールします。")
print(f"負荷軽減のため、1アクセスごとに {MIN_DELAY}〜{MAX_DELAY} 秒のランダムな遅延を設けます。")

# --- ヘルパー関数 ---

def get_title(soup):
    """BeautifulSoupオブジェクトから<title>タグのテキストを抽出"""
    title_tag = soup.find('title')
    return title_tag.string.strip() if title_tag and title_tag.string else "タイトルなし"

def normalize_url(url):
    """URLからフラグメント(#...)を除去し、正規化する"""
    parsed = urlparse(url)
    # フラグメント（#...）を除去
    normalized_url = parsed.scheme + "://" + parsed.netloc + parsed.path
    
    if parsed.params:
        normalized_url += ";" + parsed.params
    if parsed.query:
        normalized_url += "?" + parsed.query
        
    return normalized_url

def is_crawlable_link(url):
    """クロール対象として有効なURLかチェック（スキーム、同一ドメイン、静的ファイルでないか）"""
    if not url:
        return False
    
    parsed = urlparse(url)
    
    # http/https以外のスキームは除外
    if parsed.scheme and parsed.scheme not in ['http', 'https']:
        return False
        
    # 静的ファイル拡張子の除外
    if re.search(r'\.(pdf|docx?|xlsx?|pptx?|zip|rar|tar|gz|jpe?g|png|gif|mp4|mp3|css|js|ico|xml|txt)$', parsed.path, re.IGNORECASE):
        return False
        
    # 同一ドメイン（またはサブドメイン）のみを対象とする
    if not parsed.netloc.endswith(DOMAIN):
        return False
        
    return True

# --- クロールのメインループ ---

while to_crawl and (MAX_PAGES_LIMIT is None or len(sitemap) < MAX_PAGES_LIMIT):
    current_url = to_crawl.popleft() # キューからURLを取得
    
    # time.sleep() を使って負荷軽減
    sleep_time = random.uniform(MIN_DELAY, MAX_DELAY)
    time.sleep(sleep_time)

    print(f"クロール中 ({len(sitemap)}/{len(sitemap) + len(to_crawl)}): {current_url}")
    
    try:
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 (Sitemap Extractor)'
        }
        res = requests.get(current_url, headers=headers, timeout=15)
        
        # 成功したHTMLページのみを処理
        if res.status_code == 200 and 'text/html' in res.headers.get('Content-Type', ''):
            res.encoding = res.apparent_encoding
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # 辞書型変数に格納 (key: URL, value: <title>文字列)
            title = get_title(soup)
            sitemap[current_url] = title
            
            # BeautifulSoupのfind_allで<a>タグのコンテンツ（href）を全て取得
            for a_tag in soup.find_all('a', href=True):
                href = a_tag['href'].strip()
                
                # 絶対URLに変換
                absolute_url = urljoin(current_url, href)
                
                # 正規化（フラグメント除去）
                normalized_url = normalize_url(absolute_url)
                
                # 有効なURLかチェック
                if is_crawlable_link(normalized_url):
                    if normalized_url not in processed_urls:
                        to_crawl.append(normalized_url)
                        processed_urls.add(normalized_url)
            
        # else: HTMLでない場合やエラーコードの場合は、processed_urlsに追加せずスキップ
            
    except requests.exceptions.RequestException as e:
        print(f"⚠️ エラーが発生しました: {current_url} - {e}")
        
    except Exception as e:
        print(f"⚠️ 予期せぬエラー: {current_url} - {e}")

print("\n--- クロール結果 ---")
print(f"総ページ数: {len(sitemap)}")
print("--- サイトマップ辞書（URL: <title>）---")
# 辞書型変数を print() で表示する
print(sitemap)
print("------------------------------------------")

クロールを開始します: https://www.musashino-u.ac.jp
対象ドメイン: www.musashino-u.ac.jp
最大 100 ページまでクロールします。
負荷軽減のため、1アクセスごとに 1.0〜2.0 秒のランダムな遅延を設けます。
クロール中 (0/0): https://www.musashino-u.ac.jp
クロール中 (1/99): https://www.musashino-u.ac.jp/
クロール中 (2/99): https://www.musashino-u.ac.jp/access.html
クロール中 (3/107): https://www.musashino-u.ac.jp/admission/request.html
クロール中 (4/138): https://www.musashino-u.ac.jp/contact.html
クロール中 (5/138): https://www.musashino-u.ac.jp/prospective-students.html
クロール中 (6/146): https://www.musashino-u.ac.jp/students.html
クロール中 (7/154): https://www.musashino-u.ac.jp/alumni.html
クロール中 (8/156): https://www.musashino-u.ac.jp/parents.html
クロール中 (9/158): https://www.musashino-u.ac.jp/business.html
クロール中 (10/160): https://www.musashino-u.ac.jp/guide/
クロール中 (11/178): https://www.musashino-u.ac.jp/guide/profile/
クロール中 (12/203): https://www.musashino-u.ac.jp/guide/activities/
クロール中 (13/205): https://www.musashino-u.ac.jp/guide/campus/
クロール中 (14/206): https://www.musashino-u.ac.jp/guide/fa